In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib.pyplot import *
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
sns.set_style('white')
sns.set_style('ticks')
sns.set_context('talk')

In [3]:
df = pd.read_feather('vocalization_df.feather')

df_c2 = df[df.cohort == 'c2']
df_c4 = df[df.cohort == 'c4']
df_c5 = df[df.cohort == 'c5']

c2_latents = np.vstack(df_c2.latent_means.values)
c4_latents = np.vstack(df_c4.latent_means.values)
c5_latents = np.vstack(df_c5.latent_means.values)

all_latents = np.concatenate((c2_latents, c4_latents, c5_latents))

In [ ]:
train, test = train_test_split(all_latents, test_size=.25, train_size=.75)
data = train

# Create a list to store the log-likelihoods
log_likelihoods = []

# Create a list of values for the number of components
n_components = np.insert(np.arange(10, 201, 10), 0, 1)

# Loop over the number of components
for n in n_components:
    # Initialize the GMM model
    gmm = GaussianMixture(n_components=n, covariance_type='diag')
    # Train the GMM model on your data
    gmm.fit(data)
    # Store the log-likelihood of the data given the model
    log_likelihoods.append(gmm.score(test))

In [ ]:
plot(n_components, log_likelihoods, 'k')
xlabel('Number of components')
ylabel('Log-likelihood')

In [6]:
#pick a k, then train final model
n=70
gmm = GaussianMixture(n_components=n, covariance_type='diag')
data = all_latents[:, all_latents.std(0) > .75] #model the latents that account for most of the variance 
gmm.fit(data)

#get the labels
labels = gmm.predict(data)

#save labels to dataframe
df['z_70'] = labels

In [ ]:
df_c2 = df[df.cohort=='c2']
df_c4 = df[df.cohort=='c4']
df_c5 = df[df.cohort=='c5']

z = df.groupby('z_70')['timestamp'].count().values
reorder = np.argsort(z)[::-1]


z_c2 = df_c2.groupby('z_70')['timestamp'].count().values[reorder]
z_c4 = df_c4.groupby('z_70')['timestamp'].count().values[reorder]
z_c5 = df_c5.groupby('z_70')['timestamp'].count().values[reorder]

z_c2_prop = z_c2/sum(z_c2)
z_c4_prop = z_c4/sum(z_c4)
z_c5_prop = z_c5/sum(z_c5)

z_all_prop = np.array([z_c2_prop, z_c4_prop, z_c5_prop])

In [ ]:
figure(figsize=(14,3))

plot(z[reorder]/sum(z[reorder]), 'k', label='Cumulative')
plot(z_c2_prop, label='C2')
plot(z_c4_prop, label='C4')
plot(z_c5_prop, label='C5')

ylabel('Proportion usage')
xlabel('GMM Cluster (Sorted by cumulative usage)')

legend(bbox_to_anchor=(1,1))
sns.despine()
tight_layout()
# savefig('/Users/ralph/Downloads/gmm_usage.svg')

In [ ]:
figure(figsize=(18,12))

_fontsize=12

subplot(4,1,1)
plot(z[reorder]/sum(z[reorder]), 'k', label='Cumulative')
plot(z_c2_prop, label='C2')
plot(z_c4_prop, label='C4')
plot(z_c5_prop, label='C5')

ylabel('Proportion usage')
xlabel('GMM Cluster (Sorted by cumulative usage)')
xticks(np.arange(len(z_c2_prop)), fontsize=_fontsize, rotation=45)
legend(bbox_to_anchor=(1,1))
sns.despine()

subplot(4,1,2)
plot(z_c2_prop[z2z4_diff], label='C2', c='C0')
plot(z_c4_prop[z2z4_diff], label='C4', c='C1')
legend(bbox_to_anchor=(1,1))
xticks(np.arange(len(z_c2_prop)), z2z4_diff, rotation=45, fontsize=_fontsize)
ylabel('Proportion usage')
xlabel('GMM Cluster (Sorted by usage difference)')

subplot(4,1,3)
plot(z_c2_prop[z2z5_diff], label='C2', c='C0')
plot(z_c5_prop[z2z5_diff], label='C5', c='C2')
legend(bbox_to_anchor=(1,1))
xticks(np.arange(len(z_c2_prop)), z2z5_diff, rotation=45, fontsize=_fontsize)

subplot(4,1,4)
plot(z_c4_prop[z4z5_diff], label='C4', c='C1')
plot(z_c5_prop[z4z5_diff], label='C5', c='C2')
legend(bbox_to_anchor=(1,1))
xticks(np.arange(len(z_c2_prop)), z4z5_diff, rotation=45, fontsize=_fontsize)

sns.despine()
tight_layout()
# savefig('usages-sklearn.svg')